In [1]:
import os
import zipfile
import rasterio
import numpy as np
from tqdm import tqdm

In [2]:
zip_dir = "D:/code/computer-vision/active/mosaics"
output_dir = "./mosaics" 
os.makedirs(output_dir, exist_ok=True)

bands = ["B02", "B03", "B04", "B08"]

In [3]:

for zip_file in tqdm(os.listdir(zip_dir)):
    if zip_file.endswith(".zip"):
        
        zip_path = f"{zip_dir}/{zip_file}"
        extract_path = f"{zip_dir}/{os.path.splitext(zip_file)[0]}"
        output_path = f"{output_dir}/{os.path.splitext(zip_file)[0]}_rgbnir.tif"
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
                    
        tif_files = {band: None for band in bands}
        for root, _, files in os.walk(extract_path):
            for file in files:
                for band in bands:
                    if band in file and file.endswith(".tif"):
                        tif_files[band] = f"{root}/{file}"

        with rasterio.open(tif_files["B02"]) as src:
            meta = src.meta.copy()
            meta.update(count=len(bands))

        with rasterio.open(output_path, "w", **meta) as dest:
            for idx, band in enumerate(bands, start=1):
                with rasterio.open(tif_files[band]) as src:
                    dest.write(src.read(1), idx)

100%|██████████| 27/27 [00:00<?, ?it/s]
